In [1]:
import pandas as pd
pd.set_option("display.float_format", "{:.3f}".format)
pd.set_option("display.max_columns", 80)
pd.set_option("display.max_rows", 80)
import numpy as np

# Visualization
import matplotlib.pyplot as plt
plt.style.use("fivethirtyeight")
%matplotlib inline
import seaborn as sns
sns.set_style("whitegrid")

# Data preprocessing
from sklearn.preprocessing import RobustScaler

# Helper functions
from helper_functions import handle_missing_values

This notebook will contain **data preprocessing steps** before building a machine learning model for the binary classification task.

In [2]:
df_train = pd.read_csv('../data/training2.csv')
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3700 entries, 0 to 3699
Data columns (total 18 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   v1          3661 non-null   object 
 1   v2          3661 non-null   float64
 2   v3          3700 non-null   float64
 3   v4          3636 non-null   object 
 4   v5          3700 non-null   float64
 5   v6          3700 non-null   float64
 6   v7          3700 non-null   float64
 7   v8          3700 non-null   object 
 8   v9          3700 non-null   object 
 9   v10         3700 non-null   int64  
 10  v11         3700 non-null   object 
 11  v12         3700 non-null   object 
 12  v13         3600 non-null   float64
 13  v14         3700 non-null   int64  
 14  v15         3600 non-null   float64
 15  v16         1555 non-null   object 
 16  v17         3700 non-null   int64  
 17  classLabel  3700 non-null   object 
dtypes: float64(7), int64(3), object(8)
memory usage: 520.4+ KB


In [3]:
df_val = pd.read_csv('../data/validation2.csv')
df_val.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 18 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   v1          197 non-null    object 
 1   v2          197 non-null    float64
 2   v3          200 non-null    float64
 3   v4          198 non-null    object 
 4   v5          200 non-null    float64
 5   v6          200 non-null    float64
 6   v7          200 non-null    float64
 7   v8          200 non-null    object 
 8   v9          200 non-null    object 
 9   v10         200 non-null    int64  
 10  v11         200 non-null    object 
 11  v12         200 non-null    object 
 12  v13         197 non-null    float64
 13  v14         200 non-null    int64  
 14  v15         197 non-null    float64
 15  v16         89 non-null     object 
 16  v17         200 non-null    int64  
 17  classLabel  200 non-null    object 
dtypes: float64(7), int64(3), object(8)
memory usage: 28.3+ KB


#### **1.0 Dropping problematic columns**

**More than half of the rows in feature** `v16` **are missing** in both `df_train` & `df_val`. Using imputation techniques may not be the best approach because imputing missing values in a categorical variable with such a high rate of missing data can introduce bias and distort the original distribution. Hence, **we'll be dropping this column from both datasets**.

We've also identified that **columns** `v13` **&** `v15` **are perfectly correlated with each other**. Hence, we'll randomly drop one of them. In this case, we'll also drop `v15`. Should information on these features be available, we may reconsider this.

`v17` is also dropped due to having suspiciously high correlation with the target, `classLabel`.

In [4]:
df_train.drop(['v16', 'v15', 'v17'], inplace=True, axis='columns')
df_val.drop(['v16', 'v15', 'v17'], inplace=True, axis='columns')

#### **2.0 Impute missing values**

The rest of the features have a smaller portion of missing values.

For numerical columns (`v2`, `v13` & `v15`), I will impute the missing values with the median value as the median is less impacted by outliers compared to the mean since most of the numerical columns are positively skewed with outliers.

For categorical columns (`v1` & `v4`), I will impute the missing category by labelling the missing value with their respective modes. This is due to both columns having clear modes and imputing with the mode helps maintain the original distribution of the categorical variable.

In [5]:
missing_percentage_df_train = df_train.isnull().sum() * 100/len(df_train)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(missing_percentage_df_train)

v1           1.054
v2           1.054
v3           0.000
v4           1.730
v5           0.000
v6           0.000
v7           0.000
v8           0.000
v9           0.000
v10          0.000
v11          0.000
v12          0.000
v13          2.703
v14          0.000
classLabel   0.000
dtype: float64


In [6]:
missing_percentage_df_val = df_val.isnull().sum() * 100/len(df_val)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(missing_percentage_df_val)

v1           1.500
v2           1.500
v3           0.000
v4           1.000
v5           0.000
v6           0.000
v7           0.000
v8           0.000
v9           0.000
v10          0.000
v11          0.000
v12          0.000
v13          1.500
v14          0.000
classLabel   0.000
dtype: float64


In [7]:
# separate categorical & numerical columns for analyses
CatCols=['v1', 'v4', 'v8', 'v9', 'v11',
         'v12', 'classLabel']
NumCols=list(set(df_train.columns)-set(CatCols))

In [8]:
# impute using the helper function we imported at the start
df_train2 = handle_missing_values(df_train, categorical_cols=CatCols, numerical_cols=NumCols)
df_val2 = handle_missing_values(df_val, categorical_cols=CatCols, numerical_cols=NumCols)

In [9]:
df_train2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3700 entries, 0 to 3699
Data columns (total 15 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   v1          3700 non-null   category
 1   v2          3700 non-null   float64 
 2   v3          3700 non-null   float64 
 3   v4          3700 non-null   category
 4   v5          3700 non-null   float64 
 5   v6          3700 non-null   float64 
 6   v7          3700 non-null   float64 
 7   v8          3700 non-null   category
 8   v9          3700 non-null   category
 9   v10         3700 non-null   int64   
 10  v11         3700 non-null   category
 11  v12         3700 non-null   category
 12  v13         3700 non-null   float64 
 13  v14         3700 non-null   int64   
 14  classLabel  3700 non-null   category
dtypes: category(7), float64(6), int64(2)
memory usage: 257.5 KB


In [10]:
# final check
df_val2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 15 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   v1          200 non-null    category
 1   v2          200 non-null    float64 
 2   v3          200 non-null    float64 
 3   v4          200 non-null    category
 4   v5          200 non-null    float64 
 5   v6          200 non-null    float64 
 6   v7          200 non-null    float64 
 7   v8          200 non-null    category
 8   v9          200 non-null    category
 9   v10         200 non-null    int64   
 10  v11         200 non-null    category
 11  v12         200 non-null    category
 12  v13         200 non-null    float64 
 13  v14         200 non-null    int64   
 14  classLabel  200 non-null    category
dtypes: category(7), float64(6), int64(2)
memory usage: 14.9 KB


#### **3.0 Binning categorical columns with high cardinality**

As previously identified, **different values for** `v4` **(l) and** `v12` **(o) have very small occurences in the dataset (<1%)**. Hence, we will use binning (where a smaller variable is subsumed into a larger variable) to reduce the feature's cardinality.

However, since I am not privy to the real-world nature of each feature, I will subsume the smallest occuring value in the non-largest occuring value e.g., *l* in `v4` will be subsumed into `y` instead of `u`. We ought to consider domain specific knowledge of course should it be available in the future.

I'd like to also note that value *p* in `v12` is especially sparse (<2%). While an argument can be made to bin it as well, I will leave it be for now due to the lack of domain knowledge (since *p*'s sparsity may be justified in real life!).

In [11]:
# initialize dictionaries to remap values in columns v4 & v12
bin_v4 = {
    'u': 'u',
    'y': 'y',
    'l': 'y',
}

bin_v12 = {
    'g': 'g',
    's': 's',
    'p': 'p',
    'o': 'p',
}

In [12]:
print(f"Unique values in v4 before binning: {df_train2.v4.unique()}")
print("")

df_train2.v4.replace(bin_v4, inplace=True)

print(f"Unique values in v4 after binning: {df_train2.v4.unique()}")

Unique values in v4 before binning: ['u', 'y', 'l']
Categories (3, object): ['l', 'u', 'y']

Unique values in v4 after binning: ['u', 'y']
Categories (2, object): ['u', 'y']


In [13]:
print(f"Unique values in v12 before binning: {df_val2.v12.unique()}")
print("")

df_val2.v12.replace(bin_v12, inplace=True)

print(f"Unique values in v12 after binning: {df_val2.v12.unique()}")

Unique values in v12 before binning: ['s', 'g', 'p', 'o']
Categories (4, object): ['g', 'o', 'p', 's']

Unique values in v12 after binning: ['s', 'g', 'p']
Categories (3, object): ['g', 'p', 's']


#### **4.0 One-Hot encoding categorical columns**

Next I will transform the categorical features to one-hot-encoded features. This is because a logistic regression model (our choice for a baseline) can only take in numeric input. An argument can be made for label encoding but we do not know if there's an ordinal structure for any of the columns.

In [14]:
print(f"List of categorical columns:\n{CatCols}")
print("")
print(f"List of numerical columns:\n{NumCols}")

List of categorical columns:
['v1', 'v4', 'v8', 'v9', 'v11', 'v12', 'classLabel']

List of numerical columns:
['v10', 'v2', 'v7', 'v13', 'v14', 'v6', 'v5', 'v3']


In [15]:
max_number_of_categories = max([df_train2[col].nunique() for col in CatCols])
print(f"Max number of categories in categorical features: {max_number_of_categories}")

Max number of categories in categorical features: 3


In [16]:
df_train2 = pd.get_dummies(
    df_train2,
    columns=CatCols,
    dtype=int,
    drop_first=True # to avoid multi-collinearity
)
df_train2.head(3)

,v2,v3,v5,v6,v7,v10,v13,v14,v1_b,v4_y,v8_t,v9_t,v11_t,v12_p,v12_s,classLabel_yes.
0,17.920,0.000,-0.840,0.523,1.750,1,80.000,5,0,0,0,1,1,0,0,0
1,16.920,0.000,-2.160,0.774,0.290,0,200.000,0,1,1,0,0,0,0,1,0
2,31.250,0.000,1.751,0.760,0.000,1,96.000,19,1,0,0,1,0,0,0,0


In [17]:
df_val2 = pd.get_dummies(
    df_val2,
    columns=CatCols,
    dtype=int,
    drop_first=True # to avoid multi-collinearity
)
df_val2.head(3)

,v2,v3,v5,v6,v7,v10,v13,v14,v1_b,v4_y,v8_t,v9_t,v11_t,v12_p,v12_s,classLabel_yes.
0,32.330,0.001,0.840,0.545,1.585,0,420.000,0,1,0,1,0,1,0,1,0
1,23.580,0.000,-4.174,0.864,0.540,0,136.000,1,1,0,0,0,1,0,0,0
2,36.420,0.000,2.232,0.627,0.585,0,240.000,3,1,1,0,0,0,0,0,0


#### **5.0 Scaling on numerical columns**

In [18]:
print(NumCols)

['v10', 'v2', 'v7', 'v13', 'v14', 'v6', 'v5', 'v3']


Our choice of scaler is the `RobustScaler()`. This scaler is used to scale features while taking into account the presence of outliers in your data. It scales the features to be centered around the median and within a certain interquartile range, which makes it robust to the influence of outliers. This is especially the case since most of our numerical features are positively skewed. 

However, it is important to note that **features** `v5` **&** `v6` **are not skewed positively**. Hence, later iterations can explore different scaling methods on them e.g., MinMax Scaler, Standard Scaler, etc.

In [19]:
scaler = RobustScaler()

df_train2[NumCols] = scaler.fit_transform(df_train2[NumCols])
df_val2[NumCols] = scaler.fit_transform(df_val2[NumCols])

In [20]:
df_train2.head(3)

,v2,v3,v5,v6,v7,v10,v13,v14,v1_b,v4_y,v8_t,v9_t,v11_t,v12_p,v12_s,classLabel_yes.
0,-0.632,-0.457,-0.160,-1.448,0.000,-0.167,-0.146,-0.102,0,0,0,1,1,0,0,0
1,-0.691,-0.482,-0.428,-0.075,-0.324,-0.333,0.292,-0.107,1,1,0,0,0,0,1,0
2,0.152,-0.385,0.366,-0.150,-0.389,-0.167,-0.088,-0.089,1,0,0,1,0,0,0,0


In [21]:
df_val2.head(3)

,v2,v3,v5,v6,v7,v10,v13,v14,v1_b,v4_y,v8_t,v9_t,v11_t,v12_p,v12_s,classLabel_yes.
0,0.163,0.728,0.309,-0.600,0.225,0.000,1.300,-0.009,1,0,1,0,1,0,1,0
1,-0.372,-0.121,-0.779,0.641,-0.209,0.000,-0.120,-0.007,1,0,0,0,1,0,0,0
2,0.413,-0.276,0.611,-0.279,-0.190,0.000,0.400,-0.003,1,1,0,0,0,0,0,0


#### **6.0 Export cleaned dataset for modeling**

In [22]:
df_train2.to_csv("../data/processed_training_nov17.csv", index=False)
df_val2.to_csv("../data/processed_validation_nov17.csv", index=False)